In [3]:
import pandas as pd
import matplotlib as plt
import numpy as np
import math

In [117]:
# Global Functions
def sqScore(hits, shots, target):
    score = round(hits * math.sqrt((hits/shots)*100))
    print(f'Score: {score}')
    misses = shots - hits
    i = 0

    # Hits +1
    while target > score: 
        i += 1
        score = (hits + i) * math.sqrt(((hits + i)/shots)*100)
    else:
        print(f'Converted Hits: {i}')    
        score = hits * math.sqrt((hits/shots)*100)
        i = 0
    
    # Hits, Shots +1
    while target > score: 
        i += 1
        score = (hits + i) * math.sqrt(((hits + i)/(shots + i))*100)
    else:
        print(f'New Hits: {i}')    
        score = hits * math.sqrt((hits/shots)*100)
        i = 0

    # Shots -1
    while target > score: 
        i += 1
        score = (hits) * math.sqrt((hits/(shots - i))*100)
        if (shots - i) < hits:
            score = hits * math.sqrt((hits/shots)*100)
            break
    else:
        print(f'Fewer Misses: {i}')    
        score = hits * math.sqrt((hits/shots)*100)
        i = 0

    tableShots = [x for x in range(int(target/10), (round(1.25 * (target/10))))]
    tableHits = [math.ceil((target/10)**(2/3) * x**(1./3)) for x in tableShots]
    
    targetTable = pd.DataFrame({"Hits": tableHits, "Shots": tableShots})
    targetTable = targetTable.assign(Accuracy=100 * round(targetTable["Hits"]/targetTable["Shots"], 3)).set_index("Shots")
    print(targetTable)

sqScore(131, 161, 1490)

Score: 1182
Converted Hits: 22
New Hits: 32
       Hits  Accuracy
Shots                
149     149     100.0
150     150     100.0
151     150      99.3
152     150      98.7
153     151      98.7
154     151      98.1
155     151      97.4
156     152      97.4
157     152      96.8
158     152      96.2
159     153      96.2
160     153      95.6
161     153      95.0
162     154      95.1
163     154      94.5
164     154      93.9
165     155      93.9
166     155      93.4
167     155      92.8
168     156      92.9
169     156      92.3
170     156      91.8
171     156      91.2
172     157      91.3
173     157      90.8
174     157      90.2
175     158      90.3
176     158      89.8
177     158      89.3
178     159      89.3
179     159      88.8
180     159      88.3
181     159      87.8
182     160      87.9
183     160      87.4
184     160      87.0
185     161      87.0
